# Testing Model
---
Tahap ini melakukan pengetesan untuk model machine learning yang sudah kita buat dan menyimpan log hasil pelatihan nya dalam format json

## Import Library yang dibutuhkan 

In [19]:
import numpy as np  # library untuk komputasi numerik 
import pandas as pd # library untuk pengolahan dataframe 
import json         # library untuk data dalam format json 
import joblib       # library untuk load model dalam format joblib 
import unittest     # library untuk melakukan testing program 

# library untuk pengukuran model machine learning 
from sklearn.metrics import accuracy_score 

## Membuat Augmentasi untuk menguji model
--- 
Membuat data dummy dalam bentuk array secara acak untuk melatih model machine learning kita yang sudah dilatih sebelmunya 

In [20]:
# Membuat data baru untuk pengujian
new_data = np.array([
    [1, 170.5, 60.3],  # Gender, Height, Weight
    [0, 180.1, 85.7],
    [1, 151.0, 78.1],
    [0, 165.3, 70.0],
    [1, 175.2, 80.5],
    [0, 150.4, 45.3],
    [1, 160.7, 55.8],
    [0, 190.0, 95.2],
    [1, 168.8, 75.4],
    [0, 178.5, 65.6],
])

## Test model dengan unittest
---
Setelah membuat data mentah dalam format array, selanjutnya load file model dan buat variabel new labels serta akurasi minimum nya 

In [22]:
# Path ke file model 
MODEL_PATH = '../models/voting_classifier_model.joblib'

# Label untuk data baru (hanya untuk evaluasi)
new_labels = np.array([2, 3, 3, 3, 3, 2, 2, 3, 3, 2])  # Contoh label ground truth

# Ambang batas akurasi minimum
MIN_ACCURACY = 0.8

## Buat Class OOP untuk unittest 
---
Membuat modul testing dengan menggunakan pendekatan oop 

In [25]:
class TestVotingClassifier(unittest.TestCase):
    """Unittest untuk model VotingClassifier."""

    @classmethod
    def setUpClass(cls):
        """Memuat model sekali untuk semua pengujian."""
        try:
            cls.model = joblib.load(MODEL_PATH)
        except FileNotFoundError:
            raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")

    def test_model_file_exists(self):
        """Test: Memastikan file model ada."""
        try:
            joblib.load(MODEL_PATH)
        except FileNotFoundError:
            self.fail(f"Model file not found at {MODEL_PATH}")

    def test_model_predictions_within_classes(self):
        """Test: Memastikan prediksi sesuai dengan kelas yang diharapkan."""
        predictions = self.model.predict(new_data)
        # Periksa apakah prediksi dalam rentang kelas
        self.assertTrue(np.all(predictions >= 0), "Predictions contain invalid classes below 0")
        self.assertTrue(np.all(predictions <= 5), "Predictions contain invalid classes above 5")

    def test_model_accuracy(self):
        """Test: Memastikan akurasi model memenuhi ambang batas."""
        predictions = self.model.predict(new_data)
        accuracy = accuracy_score(new_labels, predictions)
        self.assertGreaterEqual(accuracy, MIN_ACCURACY, f"Model accuracy {accuracy:.4f} is below the minimum threshold {MIN_ACCURACY}")

    def test_model_handles_new_data(self):
        """Test: Memastikan model dapat menangani data baru."""
        try:
            predictions = self.model.predict(new_data)
        except Exception as e:
            self.fail(f"Model failed to handle new data: {e}")
        # Periksa jumlah prediksi sama dengan jumlah data input
        self.assertEqual(len(predictions), len(new_data), "Number of predictions does not match the input data size")

    def test_model_probability_output(self):
        """Test: Memastikan model memberikan probabilitas untuk prediksi."""
        try:
            probabilities = self.model.predict_proba(new_data)
        except Exception as e:
            self.fail(f"Model failed to return probabilities: {e}")
        # Periksa apakah probabilitas memiliki dimensi yang benar
        self.assertEqual(probabilities.shape, (len(new_data), 6), "Probability output shape is incorrect")


# Menjalankan unittest tanpa konflik dengan Jupyter Notebook
if __name__ == "__main__":
    # Membuat suite pengujian
    suite = unittest.TestLoader().loadTestsFromTestCase(TestVotingClassifier)
    
    # Menjalankan pengujian
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

test_model_accuracy (__main__.TestVotingClassifier.test_model_accuracy)
Test: Memastikan akurasi model memenuhi ambang batas. ... ok
test_model_file_exists (__main__.TestVotingClassifier.test_model_file_exists)
Test: Memastikan file model ada. ... ok
test_model_handles_new_data (__main__.TestVotingClassifier.test_model_handles_new_data)
Test: Memastikan model dapat menangani data baru. ... ok
test_model_predictions_within_classes (__main__.TestVotingClassifier.test_model_predictions_within_classes)
Test: Memastikan prediksi sesuai dengan kelas yang diharapkan. ... ok
test_model_probability_output (__main__.TestVotingClassifier.test_model_probability_output)
Test: Memastikan model memberikan probabilitas untuk prediksi. ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.503s

OK


## Simpan hasil pengetesan dalam format json 
---
Setelah berhasil melakukan testing simpan file log dalam format json 

In [29]:
import os

# Menjalankan unittest dan menyimpan hasil ke JSON
if __name__ == "__main__":
    # Membuat suite pengujian
    suite = unittest.TestLoader().loadTestsFromTestCase(TestVotingClassifier)

    # Menjalankan pengujian
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    
    # Menyiapkan data hasil pengujian untuk JSON
    test_results = {
        "total_tests": result.testsRun,
        "failures": [
            {"test": str(f[0]), "reason": str(f[1])} for f in result.failures
        ],
        "errors": [
            {"test": str(e[0]), "reason": str(e[1])} for e in result.errors
        ],
        "skipped": [
            {"test": str(s[0]), "reason": str(s[1])} for s in result.skipped
        ],
        "successful_tests": result.testsRun - len(result.failures) - len(result.errors) - len(result.skipped),
    }
    
    # Menyimpan hasil ke file JSON
    with open("../logs/unittest_results.json", "w") as json_file:
        json.dump(test_results, json_file, indent=4)

    print("Hasil pengujian telah disimpan di 'logs/unittest_results.json'")


test_model_accuracy (__main__.TestVotingClassifier.test_model_accuracy)
Test: Memastikan akurasi model memenuhi ambang batas. ... ok
test_model_file_exists (__main__.TestVotingClassifier.test_model_file_exists)
Test: Memastikan file model ada. ... ok
test_model_handles_new_data (__main__.TestVotingClassifier.test_model_handles_new_data)
Test: Memastikan model dapat menangani data baru. ... ok
test_model_predictions_within_classes (__main__.TestVotingClassifier.test_model_predictions_within_classes)
Test: Memastikan prediksi sesuai dengan kelas yang diharapkan. ... ok
test_model_probability_output (__main__.TestVotingClassifier.test_model_probability_output)
Test: Memastikan model memberikan probabilitas untuk prediksi. ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.601s

OK


Hasil pengujian telah disimpan di 'logs/unittest_results.json'
